## __Import__

In [2]:
import pandas as pd
import numpy as np
from dataloader import Dataloader
from tqdm import tqdm
from datetime import datetime
from statsbombpy import sb
from collections import Counter

dataloader = Dataloader()
df = dataloader.get_dimension(dimension="standard_stats",row_filter=False)
df.columns

[2025-03-19 17:14:15] Loading data form local file system


Index(['player', 'match_id', 'minute', 'substitution_replacement',
       'substitution_outcome'],
      dtype='object')

In [3]:
all_players = set()
for player in df.loc[df["player"].notna(), "player"].values:
    all_players.add(player)

for player in df.loc[df["substitution_replacement"].notna(),"substitution_replacement"].values:
    all_players.add(player)
print(f"In total {len(all_players)} players applapeared on the pitch during the season.")

In total 478 players applapeared on the pitch during the season.


## Main Method

In [4]:
def log_step(message):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"[{timestamp}] {message}")


def get_most_frequent_pos(input_list):
    input_list = extract_positions(input_list, unique=False)
    try:
        counter = Counter(input_list)
        max_count = max(counter.values())
        most_frequent_pos = [item for item, count in counter.items() if count == max_count]
    except:
        most_frequent_pos = None
    return most_frequent_pos

def extract_positions(positions,unique=False):
    # Flatten the array of lists into a single list
    flattened_positions = np.concatenate(positions.values).tolist()
    # Check if the entire flattened list is empty
    if all(len(pos_list) == 0 for pos_list in flattened_positions):
        return []
    # Extract positions from the dictionaries

    if unique:
        positions = [pos["position"] for pos in flattened_positions if "position" in pos]
        return list(set(positions))
        
    else:
        return [pos["position"] for pos in flattened_positions if "position" in pos]

def get_positions_played(df):
    positions_played_df = df.groupby("player_name").agg(
        positions_played=("positions", lambda x: extract_positions(x, unique=True)),
        position=("positions", lambda x: get_most_frequent_pos(x)) 
    )
    return positions_played_df

def get_minutes_played(df):
    df_with_flags = df.copy()

    all_players = set()
    match_duration = df_with_flags["minute"].max()

    for player in df_with_flags.loc[df_with_flags["player"].notna(),"player"].values:
        all_players.add(player)

    for player in df_with_flags.loc[df_with_flags["substitution_replacement"].notna(), "substitution_replacement"].values:
        all_players.add(player)

    # flags 
    df_with_flags["is_substituted"] = df_with_flags["substitution_outcome"].notna()
    df_with_flags["match_duration"] = match_duration
    df_with_flags["minutes_played_subbed_in"] = match_duration - df_with_flags["minute"]

    # filter
    df_player_subbed_out = df_with_flags.loc[df_with_flags["is_substituted"]==True,["player", "minute"]]
    df_player_subbed_in = df_with_flags.loc[df_with_flags["is_substituted"]==True, ["substitution_replacement", "minutes_played_subbed_in"]]
    
    # rename
    df_player_subbed_in = df_player_subbed_in.rename({"substitution_replacement":"player",
                                                      "minutes_played_subbed_in" :"minutes_played"}, axis=1)
    df_player_subbed_out = df_player_subbed_out.rename({"minute":"minutes_played"}, axis=1)

    # add subbed_in subbed_of mask
    df_player_subbed_in["subbed_in"] = 1
    df_player_subbed_out["subbed_out"] = 1

    df_subbed_player = pd.concat([df_player_subbed_in, df_player_subbed_out],axis=0)

    # add remaining players that were not subbed off
    players_not_subbed_off = [player for player in all_players if player not in df_subbed_player["player"].values]
    df_player_not_subbed_off = pd.DataFrame({"player": players_not_subbed_off, "minutes_played" : 90})

    df_result = pd.concat([df_subbed_player, df_player_not_subbed_off],axis=0).reset_index(drop=True)
    df_result["match_played"] = 1

    return df_result 

def analyze_standard_stats(df):
    match_ids = df["match_id"].unique()
    columns = ["player", "match_id", "minute", "substitution_replacement", "substitution_outcome"]
    standard_stats = pd.DataFrame()
    
    log_step("Calculating match_played and minutes_played")
    concated_matches_df = pd.DataFrame()

    for match_id in tqdm(match_ids, desc="Concatenating matches", unit="match"):
        current = get_minutes_played(df.loc[df["match_id"]==match_id,columns])
        concated_matches_df = pd.concat([concated_matches_df, current])

    df_match_minutes_played = concated_matches_df.groupby("player").agg(
            match_played=("match_played","sum"),
            minutes_played=("minutes_played","sum"),
            subbed_in=("subbed_in","sum"),
            subbed_out=("subbed_out","sum")
        )
    
    log_step("Retrieve Lineups from API to map team, country, and positions_played")
    df_team_country = pd.DataFrame()
    # load and stack matches 
    for match_id in tqdm(match_ids, desc="Retrieving Lineups per game from API", unit="match"):
        lineups = sb.lineups(match_id=match_id) # ["Hertha Berlin"]
        
        for team in lineups.keys():
            player_information = lineups[team].loc[:, ["player_name","country","positions"]]
            player_information["team"] = team
            df_team_country = pd.concat([df_team_country, player_information], axis=0)
            
    df_team_country = df_team_country.drop_duplicates(subset=['player_name'])

    log_step("Process positions")
    positions_played_df = get_positions_played(df_team_country)

    log_step("Merge to final dataframe")
    # merge df_match_minutes_played and df_team_country to standard_stats
    standard_stats = pd.merge(left=df_match_minutes_played, right=df_team_country,left_on="player",right_on="player_name", how="right")

    # merge player position to standard stats
    standard_stats = pd.merge(left=standard_stats, right=positions_played_df ,left_on="player_name",right_on="player_name", how="right")
    
    # reorder columns and keep only relevant
    standard_stats = standard_stats[["player_name", "country","team","positions_played","match_played","minutes_played"]]
    
    return standard_stats

result_df = analyze_standard_stats(df)
result_df

[2025-03-19 17:14:30] Calculating match_played and minutes_played


Concatenating matches: 100%|██████████| 306/306 [00:13<00:00, 23.32match/s]


[2025-03-19 17:14:43] Retrieve Lineups from API to map team, country, and positions_played


Retrieving Lineups per game from API:   0%|          | 0/306 [00:00<?, ?match/s]c:\Users\Alexander Lorenz\anaconda3\envs\aifc\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
Retrieving Lineups per game from API:   0%|          | 1/306 [00:00<02:42,  1.88match/s]c:\Users\Alexander Lorenz\anaconda3\envs\aifc\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
Retrieving Lineups per game from API:   1%|          | 2/306 [00:01<02:41,  1.88match/s]c:\Users\Alexander Lorenz\anaconda3\envs\aifc\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
Retrieving Lineups per game from API:   1%|          | 3/306 [00:01<02:43,  1.85match/s]c:\Users\Alexander Lorenz\anaconda3\envs\aifc\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credential

[2025-03-19 17:17:27] Process positions
[2025-03-19 17:17:27] Merge to final dataframe


,player_name,country,team,positions_played,match_played,minutes_played
0,Aaron Hunt,Germany,Hamburger SV,"[Left Wing, Right Center Forward]",23.0,1742.0
1,Adam Hloušek,Czech Republic,VfB Stuttgart,[],7.0,425.0
2,Adelino André Vieira Freitas,Portugal,Wolfsburg,"[Right Midfield, Right Back]",26.0,1908.0
3,Admir Mehmedi,Switzerland,Bayer Leverkusen,[Right Center Forward],28.0,1373.0
4,Adnan Januzaj,Belgium,Borussia Dortmund,[],6.0,176.0
...,...,...,...,...,...,...
534,Özkan Yıldırım,Germany,Werder Bremen,[Right Wing],1.0,18.0
535,Ørjan Håskjold Nyland,Norway,Ingolstadt,[Goalkeeper],6.0,540.0
536,İlkay Gündoğan,Germany,Borussia Dortmund,[Right Center Midfield],25.0,1997.0
537,Łukasz Piszczek,Poland,Borussia Dortmund,[Right Back],20.0,1533.0


## __Store data__

In [5]:
result_df.to_csv("../data/standard_stats.csv",index=False)

# Validation

## __Minutes played, matches played__

In [10]:
columns = ["player", "match_id","minute" ,"substitution_replacement","substitution_outcome"] # "timestamp"
match_a = df.loc[(df["match_id"]==3890324), columns]
match_b = df.loc[(df["match_id"]==3890505), columns]


def get_minutes_played(df):
    df_with_flags = df.copy()

    all_players = set()
    match_duration = df_with_flags["minute"].max()

    for player in df_with_flags.loc[df_with_flags["player"].notna(),"player"].values:
        all_players.add(player)

    for player in df_with_flags.loc[df_with_flags["substitution_replacement"].notna(), "substitution_replacement"].values:
        all_players.add(player)

    # flags 
    df_with_flags["is_substituted"] = df_with_flags["substitution_outcome"].notna()
    df_with_flags["match_duration"] = match_duration
    df_with_flags["minutes_played_subbed_in"] = match_duration - df_with_flags["minute"]

    # filter
    df_player_subbed_out = df_with_flags.loc[df_with_flags["is_substituted"]==True,["player", "minute"]]
    df_player_subbed_in = df_with_flags.loc[df_with_flags["is_substituted"]==True, ["substitution_replacement", "minutes_played_subbed_in"]]
    
    # rename
    df_player_subbed_in = df_player_subbed_in.rename({"substitution_replacement":"player",
                                                      "minutes_played_subbed_in" :"minutes_played"}, axis=1)
    df_player_subbed_out = df_player_subbed_out.rename({"minute":"minutes_played"}, axis=1)

    # add subbed_in subbed_of mask
    df_player_subbed_in["subbed_in"] = 1
    df_player_subbed_out["subbed_out"] = 1

    df_subbed_player = pd.concat([df_player_subbed_in, df_player_subbed_out],axis=0)

    # add remaining players that were not subbed off
    players_not_subbed_off = [player for player in all_players if player not in df_subbed_player["player"].values]
    df_player_not_subbed_off = pd.DataFrame({"player": players_not_subbed_off, "minutes_played" : 90})

    df_result = pd.concat([df_subbed_player, df_player_not_subbed_off],axis=0).reset_index(drop=True)
    df_result["match_played"] = 1

    return df_result 

get_minutes_played(match_a)

,player,minutes_played,subbed_in,subbed_out,match_played
0,Jens Hegeler,62,1.0,NaN,1
1,Gotoku Sakai,45,1.0,NaN,1
2,Ivo Iličević,29,1.0,NaN,1
3,Alexander Baumjohann,21,1.0,NaN,1
4,Ivica Olić,16,1.0,NaN,1
5,Valentin Stocker,9,1.0,NaN,1
6,Niklas Stark,28,NaN,1.0,1
7,Matthias Ostrzolek,45,NaN,1.0,1
8,Marcelo Alfonso Díaz Rojas,61,NaN,1.0,1
9,Salomon Armand Magloire Kalou,69,NaN,1.0,1


In [7]:
matche_ids = df["match_id"].unique()
first_match = True
columns = ["player", "match_id", "minute", "substitution_replacement", "substitution_outcome"]
result = pd.DataFrame()

for match_id in matche_ids:
    current = get_minutes_played(df.loc[df["match_id"]==match_id,columns])
    result = pd.concat([result, current])

df_match_minutes_played = result.groupby("player").agg(
    match_played=("match_played","sum"),
    minutes_played=("minutes_played","sum"),
    subbed_in=("subbed_in","sum"),
    subbed_out=("subbed_out","sum")
)
df_match_minutes_played

# df_match_minutes_played = pd.merge(left=pd.DataFrame(result.groupby("player")["match_played"].sum()), right=pd.DataFrame(result.groupby("player")["minutes_played"].sum()), on="player")
# df_match_minutes_played

,match_played,minutes_played,subbed_in,subbed_out
player,,,,
Aaron Hunt,23,1742,3.0,8.0
Adam Hloušek,7,425,3.0,0.0
Adelino André Vieira Freitas,26,1908,4.0,8.0
Admir Mehmedi,28,1373,14.0,7.0
Adnan Januzaj,6,176,6.0,0.0
...,...,...,...,...
Özkan Yıldırım,1,18,1.0,0.0
Ørjan Håskjold Nyland,6,540,0.0,0.0
İlkay Gündoğan,25,1997,3.0,7.0


### __Validate minutes per game__

In [8]:
columns = ["player", "team","match_id","minute", "timestamp","substitution_replacement","substitution_outcome"]
match_a = df.loc[(df["match_id"]==3890324), columns]

def get_minutes_played_validate(df):
    df_with_flags = df.copy()
    df_with_flags = df_with_flags[df_with_flags["player"].notna()]
    df_with_flags = df_with_flags.sort_values("substitution_replacement", ascending=True)
    match_duration = df_with_flags["minute"].max()
    df_with_flags = df_with_flags.drop_duplicates(subset=['player'], keep='first')

    # flags 
    df_with_flags["is_substituted"] = df_with_flags["substitution_outcome"].notna()
    df_with_flags["match_duration"] = match_duration

    # assign all players match_duration
    df_with_flags["minutes_played"] = df_with_flags["match_duration"]
    # overwrite mp for player that has been subbed of
    df_with_flags.loc[df_with_flags["is_substituted"]==True, "minutes_played"] = df_with_flags[df_with_flags["is_substituted"]==True]["minute"]
    
    # calculate for subbed in players
    subbed_in_players = df_with_flags[df["substitution_replacement"].notna()]["substitution_replacement"].unique()
    df_with_flags["minutes_played_subbed_in"] = df_with_flags[df_with_flags["is_substituted"]==True]["match_duration"] - df_with_flags[df_with_flags["is_substituted"]==True]["minute"]
    
    return df_with_flags


get_minutes_played_validate(match_a)


KeyError: "['team', 'timestamp'] not in index"

# __Team and Country__
Output includes also players that sat on the bench the whole season

In [12]:
from statsbombpy import sb

df_team_country = pd.DataFrame()

# load and stack matches 
for match_id in matche_ids:
    lineups = sb.lineups(match_id=match_id) # ["Hertha Berlin"]
    
    for team in lineups.keys():
        player_information = lineups[team].loc[:, ["player_name","country","positions"]]
        player_information["team"] = team
        df_team_country = pd.concat([df_team_country, player_information], axis=0)
        
df_team_country = df_team_country.drop_duplicates(subset=['player_name']) 
df_team_country

c:\Users\Alexander Lorenz\anaconda3\envs\aifc\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Alexander Lorenz\anaconda3\envs\aifc\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Alexander Lorenz\anaconda3\envs\aifc\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Alexander Lorenz\anaconda3\envs\aifc\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Alexander Lorenz\anaconda3\envs\aifc\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\Alexander Lorenz\anaconda3\envs\aifc\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarni

,player_name,country,positions,team
0,Leroy Sané,Germany,"[{'position_id': 12, 'position': 'Right Midfie...",Schalke 04
1,Jean-Eric Maxim Choupo-Moting,Cameroon,"[{'position_id': 16, 'position': 'Left Midfiel...",Schalke 04
2,Joël Andre Job Matip,Cameroon,"[{'position_id': 3, 'position': 'Right Center ...",Schalke 04
3,Sead Kolašinac,Bosnia and Herzegovina,"[{'position_id': 6, 'position': 'Left Back', '...",Schalke 04
4,Pierre-Emile Højbjerg,Denmark,[],Schalke 04
...,...,...,...,...
0,Kevin De Bruyne,Belgium,"[{'position_id': 19, 'position': 'Center Attac...",Wolfsburg
2,Ivan Perišić,Croatia,"[{'position_id': 21, 'position': 'Left Wing', ...",Wolfsburg
2,Takashi Inui,Japan,"[{'position_id': 16, 'position': 'Left Midfiel...",Eintracht Frankfurt
10,Matija Nastasić,Serbia,"[{'position_id': 5, 'position': 'Left Center B...",Schalke 04


# 1. Merge

In [ ]:
player_stats = pd.merge(left=df_match_minutes_played, right=df_team_country,left_on="player",right_on="player_name", how="right")
player_stats

,match_played,minutes_played,player_name,country,positions,team
0,34.0,2437.0,Leroy Sané,Germany,"[{'position_id': 12, 'position': 'Right Midfie...",Schalke 04
1,28.0,1883.0,Jean-Eric Maxim Choupo-Moting,Cameroon,"[{'position_id': 16, 'position': 'Left Midfiel...",Schalke 04
2,34.0,3060.0,Joël Andre Job Matip,Cameroon,"[{'position_id': 3, 'position': 'Right Center ...",Schalke 04
3,23.0,1583.0,Sead Kolašinac,Bosnia and Herzegovina,"[{'position_id': 6, 'position': 'Left Back', '...",Schalke 04
4,23.0,1310.0,Pierre-Emile Højbjerg,Denmark,[],Schalke 04
...,...,...,...,...,...,...
534,2.0,164.0,Kevin De Bruyne,Belgium,"[{'position_id': 19, 'position': 'Center Attac...",Wolfsburg
535,2.0,163.0,Ivan Perišić,Croatia,"[{'position_id': 21, 'position': 'Left Wing', ...",Wolfsburg
536,1.0,45.0,Takashi Inui,Japan,"[{'position_id': 16, 'position': 'Left Midfiel...",Eintracht Frankfurt
537,1.0,16.0,Matija Nastasić,Serbia,"[{'position_id': 5, 'position': 'Left Center B...",Schalke 04


## __Positions played__

In [15]:
from collections import Counter

def get_most_frequent_pos(input_list):
    input_list = extract_positions(input_list, unique=False)
    try:
        counter = Counter(input_list)
        max_count = max(counter.values())
        most_frequent_pos = [item for item, count in counter.items() if count == max_count]
    except:
        most_frequent_pos = None
    return most_frequent_pos

def extract_positions(positions,unique=False):
    # Flatten the array of lists into a single list
    flattened_positions = np.concatenate(positions.values).tolist()
    # Check if the entire flattened list is empty
    if all(len(pos_list) == 0 for pos_list in flattened_positions):
        return []
    # Extract positions from the dictionaries

    if unique:
        positions = [pos["position"] for pos in flattened_positions if "position" in pos]
        return list(set(positions))
        
    else:
        return [pos["position"] for pos in flattened_positions if "position" in pos]

def get_positions_played(df):
    positions_played_df = df.groupby("player_name").agg(
        positions_played=("positions", lambda x: extract_positions(x, unique=True)),
        position=("positions", lambda x: get_most_frequent_pos(x)),
        raw=("positions", lambda x: x)
    )
    return positions_played_df

positions_played_df = get_positions_played(df_team_country)
positions_played_df

,positions_played,position,raw
player_name,,,
Aaron Hunt,"[Left Wing, Right Center Forward]","[Right Center Forward, Left Wing]","[{'position_id': 22, 'position': 'Right Center..."
Adam Hloušek,[],None,[]
Adelino André Vieira Freitas,"[Right Midfield, Right Back]",[Right Midfield],"[{'position_id': 12, 'position': 'Right Midfie..."
Admir Mehmedi,[Right Center Forward],[Right Center Forward],"[{'position_id': 22, 'position': 'Right Center..."
Adnan Januzaj,[],None,[]
...,...,...,...
Özkan Yıldırım,[Right Wing],[Right Wing],"[{'position_id': 17, 'position': 'Right Wing',..."
Ørjan Håskjold Nyland,[Goalkeeper],[Goalkeeper],"[{'position_id': 1, 'position': 'Goalkeeper', ..."
İlkay Gündoğan,[Right Center Midfield],[Right Center Midfield],"[{'position_id': 13, 'position': 'Right Center..."


In [16]:
df_team_country

,player_name,country,positions,team
0,Leroy Sané,Germany,"[{'position_id': 12, 'position': 'Right Midfie...",Schalke 04
1,Jean-Eric Maxim Choupo-Moting,Cameroon,"[{'position_id': 16, 'position': 'Left Midfiel...",Schalke 04
2,Joël Andre Job Matip,Cameroon,"[{'position_id': 3, 'position': 'Right Center ...",Schalke 04
3,Sead Kolašinac,Bosnia and Herzegovina,"[{'position_id': 6, 'position': 'Left Back', '...",Schalke 04
4,Pierre-Emile Højbjerg,Denmark,[],Schalke 04
...,...,...,...,...
0,Kevin De Bruyne,Belgium,"[{'position_id': 19, 'position': 'Center Attac...",Wolfsburg
2,Ivan Perišić,Croatia,"[{'position_id': 21, 'position': 'Left Wing', ...",Wolfsburg
2,Takashi Inui,Japan,"[{'position_id': 16, 'position': 'Left Midfiel...",Eintracht Frankfurt
10,Matija Nastasić,Serbia,"[{'position_id': 5, 'position': 'Left Center B...",Schalke 04


# 2. Merge

In [ ]:
player_stats = pd.merge(left=player_stats, right=positions_played_df ,left_on="player_name",right_on="player_name", how="right")
player_stats

,match_played,minutes_played,player_name,country,positions,team,positions_played,position
0,23.0,1742.0,Aaron Hunt,Germany,"[{'position_id': 22, 'position': 'Right Center...",Hamburger SV,"[Left Wing, Right Center Forward]","[Right Center Forward, Left Wing]"
1,7.0,425.0,Adam Hloušek,Czech Republic,[],VfB Stuttgart,[],None
2,26.0,1908.0,Adelino André Vieira Freitas,Portugal,"[{'position_id': 12, 'position': 'Right Midfie...",Wolfsburg,"[Right Midfield, Right Back]",[Right Midfield]
3,28.0,1373.0,Admir Mehmedi,Switzerland,"[{'position_id': 22, 'position': 'Right Center...",Bayer Leverkusen,[Right Center Forward],[Right Center Forward]
4,6.0,176.0,Adnan Januzaj,Belgium,[],Borussia Dortmund,[],None
...,...,...,...,...,...,...,...,...
534,1.0,18.0,Özkan Yıldırım,Germany,"[{'position_id': 17, 'position': 'Right Wing',...",Werder Bremen,[Right Wing],[Right Wing]
535,6.0,540.0,Ørjan Håskjold Nyland,Norway,"[{'position_id': 1, 'position': 'Goalkeeper', ...",Ingolstadt,[Goalkeeper],[Goalkeeper]
536,25.0,1997.0,İlkay Gündoğan,Germany,"[{'position_id': 13, 'position': 'Right Center...",Borussia Dortmund,[Right Center Midfield],[Right Center Midfield]
537,20.0,1533.0,Łukasz Piszczek,Poland,"[{'position_id': 2, 'position': 'Right Back', ...",Borussia Dortmund,[Right Back],[Right Back]


In [ ]:
player_stats = player_stats[["player_name", "country","team","positions_played","match_played","minutes_played"]]
player_stats

,player_name,country,team,positions_played,match_played,minutes_played
0,Aaron Hunt,Germany,Hamburger SV,"[Left Wing, Right Center Forward]",23.0,1742.0
1,Adam Hloušek,Czech Republic,VfB Stuttgart,[],7.0,425.0
2,Adelino André Vieira Freitas,Portugal,Wolfsburg,"[Right Midfield, Right Back]",26.0,1908.0
3,Admir Mehmedi,Switzerland,Bayer Leverkusen,[Right Center Forward],28.0,1373.0
4,Adnan Januzaj,Belgium,Borussia Dortmund,[],6.0,176.0
...,...,...,...,...,...,...
534,Özkan Yıldırım,Germany,Werder Bremen,[Right Wing],1.0,18.0
535,Ørjan Håskjold Nyland,Norway,Ingolstadt,[Goalkeeper],6.0,540.0
536,İlkay Gündoğan,Germany,Borussia Dortmund,[Right Center Midfield],25.0,1997.0
537,Łukasz Piszczek,Poland,Borussia Dortmund,[Right Back],20.0,1533.0


## __Store data__

In [ ]:
player_stats.to_csv("../data/standard_stats.csv",index=False)